Imports & Model Loading

In [1]:
import os
import torch
import joblib
import numpy as np
import pandas as pd
from PIL import Image
import torch.nn as nn
from torchvision import transforms
from torchvision.models import efficientnet_b0
from xgboost import XGBRegressor
import locale
locale.setlocale(locale.LC_ALL, 'en_IN')

# Load EfficientNet model for damage classification
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
effnet = efficientnet_b0(pretrained=False)
effnet.classifier[1] = nn.Linear(effnet.classifier[1].in_features, 3)
effnet.load_state_dict(torch.load("image_classification_dataset/best_model.pth", map_location=device))
effnet.to(device)
effnet.eval()

# Load XGBoost model, encoder, and scaler
xgb_model = joblib.load("price_model_xgb.pkl")
encoder = joblib.load("encoder_xgb.pkl")
scaler = joblib.load("scaler_km.pkl")


c:\Users\SHAHZOR AHMED\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\SHAHZOR AHMED\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Define Image Transform & Damage Predictor

In [2]:
# Transform for image classification
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
class_names = ['dent', 'no_damage', 'scratch']
damage_score = {'no_damage': 0, 'scratch': 1, 'dent': 2}

def classify_damage(image_path):
    img = Image.open(image_path).convert("RGB")
    img_tensor = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        output = effnet(img_tensor)
        _, pred = torch.max(output, 1)
    return class_names[pred.item()]


Define Price Prediction Function

In [3]:
def predict_price(brand, model, year, km, fuel, transmission, owners, color, front, back, left, right):
    # Predict damage for each image
    front_d = classify_damage(front)
    back_d = classify_damage(back)
    left_d = classify_damage(left)
    right_d = classify_damage(right)

    # Create tabular row
    data = {
        'Brand': [brand],
        'Model': [model],
        'Year': [int(year)],
        'kilometers Driven': scaler.transform([[float(km)]])[0],
        'Fuel Type': [fuel],
        'Transmission': [transmission],
        'Number of Owners': [int(owners)],
        'Colour': [color],
        'FrontDamage': [damage_score[front_d]],
        'BackDamage': [damage_score[back_d]],
        'LeftDamage': [damage_score[left_d]],
        'RightDamage': [damage_score[right_d]]
    }

    # Encode and predict
    df_input = pd.DataFrame(data)
    X = encoder.transform(df_input)
    price = xgb_model.predict(X)[0]
    formatted_price = locale.format_string("₹%.2f", price, grouping=True)

    # Create damage summary
    summary = f"""### 🛠️ Damage Summary
- **Front**: {front_d.capitalize()}
- **Back**: {back_d.capitalize()}
- **Left**: {left_d.capitalize()}
- **Right**: {right_d.capitalize()}"""

    return formatted_price, summary


Launch Gradio Interface

In [4]:
"""import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("## 🚗 Vehicle Resale Price Predictor")

    with gr.Row():
        with gr.Column():
            brand = gr.Textbox(label="Brand")
            model = gr.Textbox(label="Model")
            year = gr.Number(label="Year", value=2020)
            km = gr.Number(label="Kilometers Driven")
            fuel = gr.Dropdown(["Petrol", "Diesel", "CNG", "Electric"], label="Fuel Type")
            transmission = gr.Dropdown(["Manual", "Automatic"], label="Transmission")
            owners = gr.Dropdown(["1", "2", "3", "4+"], label="Number of Owners")
            color = gr.Textbox(label="Colour")

        with gr.Column():
            front = gr.Image(type="filepath", label="Front Image")
            back = gr.Image(type="filepath", label="Back Image")
            left = gr.Image(type="filepath", label="Left Image")
            right = gr.Image(type="filepath", label="Right Image")

    submit = gr.Button("Predict Resale Price")
    output_price = gr.Textbox(label="Predicted Price")
    output_summary = gr.Markdown(label="Damage Summary")

    submit.click(fn=predict_price, 
             inputs=[brand, model, year, km, fuel, transmission, owners, color, front, back, left, right],
             outputs=[output_price, output_summary])


demo.launch(share=True)
"""

'import gradio as gr\n\nwith gr.Blocks() as demo:\n    gr.Markdown("## 🚗 Vehicle Resale Price Predictor")\n\n    with gr.Row():\n        with gr.Column():\n            brand = gr.Textbox(label="Brand")\n            model = gr.Textbox(label="Model")\n            year = gr.Number(label="Year", value=2020)\n            km = gr.Number(label="Kilometers Driven")\n            fuel = gr.Dropdown(["Petrol", "Diesel", "CNG", "Electric"], label="Fuel Type")\n            transmission = gr.Dropdown(["Manual", "Automatic"], label="Transmission")\n            owners = gr.Dropdown(["1", "2", "3", "4+"], label="Number of Owners")\n            color = gr.Textbox(label="Colour")\n\n        with gr.Column():\n            front = gr.Image(type="filepath", label="Front Image")\n            back = gr.Image(type="filepath", label="Back Image")\n            left = gr.Image(type="filepath", label="Left Image")\n            right = gr.Image(type="filepath", label="Right Image")\n\n    submit = gr.Button("Predic